# Токенизација

> Токенизација је процес представљања сирових података у виду токена (најчеће бајтови података)

Од квалитета токена доста зависи и учинак самог модела
Токен је најмањи вид податка коме модел даје значење, и пошто их може бити ограничен број, јер цена модела расте значајно са повећањем самог вокабулара, избор величине токена и шта тај токен треба да обухвати је доста тежак задатак, и мале одлуке доста могу утицати на то шта ће модел разумети из података. 


### Токенизација текста

Код токенизације текста, најчешће се текст у виду стрига прво претвори у бајтове помоћу utf-8

In [2]:
text = "Здраво" 
text_bytes = text.encode("utf-8")

print(text, text_bytes)
print(len(text), len(text_bytes))


Здраво b'\xd0\x97\xd0\xb4\xd1\x80\xd0\xb0\xd0\xb2\xd0\xbe'
6 12


- Међутим овде настаје проблем, пошто је utf-8 погодан само за латинична слова, __ћирилица__ се претвори у 2 бајта и због алгоритма који користимо за спајање токена, број токена које будемо генерисали ће бити дупло већи и сами токени ће бити мање ефикасни

- Због овог разлога ћемо користити сам текст

In [8]:
text = "Здраво"
text_ints = [ord(s) for s in text]

print (text, text_ints)
print (len(text), len(text_ints))

Здраво [1047, 1076, 1088, 1072, 1074, 1086]
6 6


### Byte-Pair Encoding

Овај алгоритам и две методе које се понављају

1. Пронађи два суседна слова која се понављају најчешће

2. Замини да два пара са новим токеном

Ове две методе се понављају над текстом за тренинг токенизера док се не добије жељени број токена

In [56]:
vocabulary = {} # Вокабулар (int) -> ('char')

INPUT_FILE = "D:\Caslav\Poso\AI\EpskiGPT\data\\narodne_pesme.txt"

# Функција за учитавањње почетног скупа знакова
def create_vocabulary(text: str):
    chars = sorted(list(set(text)))
    vocab = { i:ch for i,ch in enumerate(chars) }
    encoder = { ch:i for i,ch in enumerate(chars) }
    return vocab, encoder

# Отварање фајла из ког вадимо текст
with open(INPUT_FILE, "r") as file:
    text = file.read()

vocabulary, _ = create_vocabulary(text)

print(vocabulary)

{0: '\n', 1: ' ', 2: '!', 3: "'", 4: '(', 5: ')', 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'â', 14: 'ê', 15: 'ô', 16: '̓', 17: 'Ђ', 18: 'Ј', 19: 'Љ', 20: 'Њ', 21: 'Ћ', 22: 'Џ', 23: 'А', 24: 'Б', 25: 'В', 26: 'Г', 27: 'Д', 28: 'Е', 29: 'Ж', 30: 'З', 31: 'И', 32: 'К', 33: 'Л', 34: 'М', 35: 'Н', 36: 'О', 37: 'П', 38: 'Р', 39: 'С', 40: 'Т', 41: 'У', 42: 'Ф', 43: 'Х', 44: 'Ц', 45: 'Ч', 46: 'Ш', 47: 'а', 48: 'б', 49: 'в', 50: 'г', 51: 'д', 52: 'е', 53: 'ж', 54: 'з', 55: 'и', 56: 'к', 57: 'л', 58: 'м', 59: 'н', 60: 'о', 61: 'п', 62: 'р', 63: 'с', 64: 'т', 65: 'у', 66: 'ф', 67: 'х', 68: 'ц', 69: 'ч', 70: 'ш', 71: 'ђ', 72: 'ј', 73: 'љ', 74: 'њ', 75: 'ћ', 76: 'џ', 77: '–', 78: '—', 79: '’', 80: '“', 81: '”', 82: '„'}


In [40]:
# Поделићемо тренирање Токенајзера у неколико корака

# Помоћна функција која броји понаљање узастопних токена
def count_conseq_tokens(ids, counts={}):
    """
    За дату листу интиџера, врати речник броја понављања узаступних парова
    Пример: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    Опционо ажурира већ дата пребројавања
    """
    for pair in zip(ids[:-1], ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

count_conseq_tokens([1, 2, 3, 1, 2], {(1, 2): 2})


{(1, 2): 4, (2, 3): 1, (3, 1): 1}

In [111]:
# Функција која обацује нове токене у текст
def merge(ids, pair, idx):
    """
    Замени све парове pair који се појављују у ids листи са idx
    Пример: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
    """
    new_ids = []
    i = 0
    ids_n = len(ids)
    if ids_n <= 1 or not pair:
        return ids
    while i < ids_n - 1:
        if (ids[i], ids[i+1]) == pair:
            new_ids.append(idx)
            i+=1
        else:
            new_ids.append(ids[i])
            if i == ids_n-2:
               new_ids.append(ids[i+1]) 
        i+=1
    
    return new_ids

merge([4, 1, 1, 3, 1], (1, 1), 5)


[4, 5, 3, 1]

Једна од техника која се користи код токенизације пре спајања самих слова је подела текста на логичке делове помоћу регекса.
Како би се избегли случајеви где се спајају слова и бројеви, делови текста из суседних речи, слова и знакови, итд, користе се техника поделе текста на речи и остале знакове, тако да токенизација буде логичнија. Конкретно GPT користи специфичне регексе који деле текст на логичке целине.

In [48]:
import regex as re


# the main GPT text split patterns, see
# https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""


In [55]:
# Пример за ћирилицу

primer = "Ко сме тај може, ко не зна за страх тај иде напред! - Живојин123456"

re.findall(re.compile(GPT4_SPLIT_PATTERN), primer)

['Ко',
 ' сме',
 ' тај',
 ' може',
 ',',
 ' ко',
 ' не',
 ' зна',
 ' за',
 ' страх',
 ' тај',
 ' иде',
 ' напред',
 '!',
 ' -',
 ' Живојин',
 '123',
 '456']

- Као што видите, речи су одвојене у формату размак + реч, знакови су издвојени посебно а бројви такође посебно и то највише до 3 заједно за GPT4
- Сада када смо написали неке основне функције можемо да пробамо да направимо најједноставнији токенајзер

In [113]:
# Тренирање токенајзера
# Свака епоха замени један токен као најчешће понављани пар
def train(text, vocab_size):
    # Направи почетни вокабулар
    vocab, encoder = create_vocabulary(text)
    
    # Подели на делове са регексом
    text_chunked = re.findall((re.compile(GPT4_SPLIT_PATTERN)), text)

    # Преведи текст у почетне токене
    tokens = []
    for chunk in text_chunked:
        tokens.append([encoder[slovo] for slovo in chunk])

    # За сваки регекс изброј counts понављања
    i = len(vocab.keys())
    print(i)
    while i < vocab_size:
        counts = {}

        # Извући прој понављања
        for chunk in tokens:
            counts = count_conseq_tokens(chunk, counts)
        # Извлачи највећи пар
        print(tokens)
        pair = max(counts, key=counts.get)

        # Замени најбројнији пар
        tokens = [merge(chunk, pair, i) for chunk in tokens]

        vocab[i] = (vocab[pair[0]] , vocab[pair[1]])
        
        # Повећај бројач
        i+=1
    
    # Додај специјалне токене
     
    return vocab


vocab = train("АКдјаклсдадк адкасд адхасдхассд јињј аид аидх љњфхасуф хафуха фафа уфд а сдасд,адса.даосд ач ас", 40)


19
[[3, 4, 6, 16, 5, 8, 9, 11, 6, 5, 6, 8], [0, 5, 6, 8, 5, 11, 6], [0, 5, 6, 14, 5, 11, 6, 14, 5, 11, 11, 6], [0, 16, 7, 18, 16], [0, 5, 7, 6], [0, 5, 7, 6, 14], [0, 17, 18, 13, 14, 5, 11, 12, 13], [0, 14, 5, 13, 12, 14, 5], [0, 13, 5, 13, 5], [0, 12, 13, 6], [0, 5], [0, 11, 6, 5, 11, 6], [1, 5, 6, 11, 5], [2, 6, 5, 10, 11, 6], [0, 5, 15], [0, 5, 11]]
[[3, 4, 6, 16, 5, 8, 9, 19, 5, 6, 8], [0, 5, 6, 8, 5, 19], [0, 5, 6, 14, 5, 19, 14, 5, 11, 19], [0, 16, 7, 18, 16], [0, 5, 7, 6], [0, 5, 7, 6, 14], [0, 17, 18, 13, 14, 5, 11, 12, 13], [0, 14, 5, 13, 12, 14, 5], [0, 13, 5, 13, 5], [0, 12, 13, 6], [0, 5], [0, 19, 5, 19], [1, 5, 6, 11, 5], [2, 6, 5, 10, 19], [0, 5, 15], [0, 5, 11]]
[[3, 4, 6, 16, 5, 8, 9, 19, 5, 6, 8], [20, 6, 8, 5, 19], [20, 6, 14, 5, 19, 14, 5, 11, 19], [0, 16, 7, 18, 16], [20, 7, 6], [20, 7, 6, 14], [0, 17, 18, 13, 14, 5, 11, 12, 13], [0, 14, 5, 13, 12, 14, 5], [0, 13, 5, 13, 5], [0, 12, 13, 6], [20], [0, 19, 5, 19], [1, 5, 6, 11, 5], [2, 6, 5, 10, 19], [20], [20]]
[[3, 

Сада када смо истренирали токенајзер остале су још две методе, encode() и decode()

In [121]:
# Преводи листу интиџера (токени) у стринг (оригинални текст)
tokeni = [[34], [36], [38, 19, 24], [0, 16, 7, 18, 16], [25], [25, 6, 14], [0, 17, 18, 13, 24, 26], [0, 21, 13, 12, 21], [0, 27, 27], [0, 26], [20], [0, 22], [1, 5, 6, 11, 5], [2, 6, 5, 10, 19], [20], [20]]
tokens = []
print(vocab)
for token in tokeni:
    tokens.extend(token)

def decode(tokens: list[int]):
    text:str = ""
    
    # 1) претварање токена од највећег ка најмањем [vocab_size] -> [0]

    i = len(vocab.keys()) - 1
    reverse = {v:k for k, v in vocab.items()}
    print(reverse)
    while i >= 0:
        # print(i)
        pair = vocab[i]
        j = 0

        while j < len(tokens):
            if (tokens[j]) == i:
                print(j, i)
                tokens = tokens[:j] + ([reverse[pair[0]]] + [reverse[pair[1]]] if len(pair) == 2 else [reverse[pair]]) + ([] if j==len(tokens)-1 else tokens[j+1:])
                print(tokens)
            j+=1
        
        i-=1

    for c in tokens:
        text = text + vocab[c]

    return text

decode(tokens)

{0: ' ', 1: ',', 2: '.', 3: 'А', 4: 'К', 5: 'а', 6: 'д', 7: 'и', 8: 'к', 9: 'л', 10: 'о', 11: 'с', 12: 'у', 13: 'ф', 14: 'х', 15: 'ч', 16: 'ј', 17: 'љ', 18: 'њ', 19: ('с', 'д'), 20: (' ', 'а'), 21: ('х', 'а'), 22: (('с', 'д'), 'а'), 23: ('д', 'к'), 24: (('х', 'а'), 'с'), 25: ((' ', 'а'), 'и'), 26: ('у', 'ф'), 27: ('ф', 'а'), 28: ('А', 'К'), 29: (('А', 'К'), 'д'), 30: ((('А', 'К'), 'д'), 'ј'), 31: (((('А', 'К'), 'д'), 'ј'), 'а'), 32: ((((('А', 'К'), 'д'), 'ј'), 'а'), 'к'), 33: (((((('А', 'К'), 'д'), 'ј'), 'а'), 'к'), 'л'), 34: ((((((('А', 'К'), 'д'), 'ј'), 'а'), 'к'), 'л'), (('с', 'д'), 'а')), 35: ((' ', 'а'), ('д', 'к')), 36: (((' ', 'а'), ('д', 'к')), 'а'), 37: ((' ', 'а'), 'д'), 38: (((' ', 'а'), 'д'), ('х', 'а')), 39: ((((' ', 'а'), 'д'), ('х', 'а')), ('с', 'д'))}
{' ': 0, ',': 1, '.': 2, 'А': 3, 'К': 4, 'а': 5, 'д': 6, 'и': 7, 'к': 8, 'л': 9, 'о': 10, 'с': 11, 'у': 12, 'ф': 13, 'х': 14, 'ч': 15, 'ј': 16, 'љ': 17, 'њ': 18, ('с', 'д'): 19, (' ', 'а'): 20, ('х', 'а'): 21, (('с', 'д'),

'АКдјаклсда адка адхасдхас јињј аи аидх љњфхасуф хафуха фафа уф а сда,адса.даосд а а'

In [ ]:
def encode(text: str) -> list[int]:
    tokens = []
    # 1) регекс модела

    # 2) претварање текста по вокабулару од [0] -> [vocab_size]

    return tokens